### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [2]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [3]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [4]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [5]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (not= (get-date) "2016-12-29")
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (next-date))
;(update-eval-report (get-date))
(end-order)

nil

In [7]:
(pprint/print-table (take 200 (deref portfolio-value)))


|      :date |         :tot-value | :daily-ret |            :tot-ret |     :loan |             :leverage |
|------------+--------------------+------------+---------------------+-----------+-----------------------|
| 1980-12-15 |               1000 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-16 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-17 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-18 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-19 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-22 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-23 |             1000.0 |        0.0 |                 0.0 |       0.0 |                   0.0 |
| 1980-12-24 |             1000.0 | 

| 1981-04-02 | 3151.7105538762435 |        0.0 |   1.147945338425076 |       0.0 |                   0.0 |
| 1981-04-03 |  3205.085029546172 |        0.0 |  1.1647386203003287 |       0.0 |                   0.0 |
| 1981-04-06 | 3263.0619262747837 |        0.0 |  1.1826659957561922 |       0.0 |                   0.0 |
| 1981-04-07 | 3371.4125125220626 |        0.0 |  1.2153317997299256 |       0.0 |                   0.0 |
| 1981-04-08 |  3457.878137969279 |        0.0 |  1.2406551460297461 |       0.0 |                   0.0 |
| 1981-04-09 | 3575.9014704769306 |        0.0 |  1.2742173039330558 |       0.0 |                   0.0 |
| 1981-04-10 | 3662.9356861594947 |        0.0 |  1.2982649259465235 |       0.0 |                   0.0 |
| 1981-04-13 | 3724.7393415087495 |        0.0 |  1.3149968739492353 |       0.0 |                   0.0 |
| 1981-04-14 | 3837.3780603639784 |        0.0 |  1.3447893366075365 |       0.0 |                   0.0 |
| 1981-04-15 | 3950.9864022707416 |  

| 1981-07-23 |  9786.753460332502 |        0.0 |   1.607617565691676 |   2222.25 |   0.29377341310678967 |
| 1981-07-24 |   9913.94755093735 |        0.0 |   1.611450464057507 | 2268.8125 |   0.29676552276494145 |
| 1981-07-27 | 10000.997729462182 |        0.0 |  1.6140836033014478 |    2316.0 |    0.3013663870219621 |
| 1981-07-28 |  9991.584922651178 |        0.0 |  1.6137965260845937 | 2361.9375 |   0.30957361056918475 |
| 1981-07-29 | 10016.701468320003 |        0.0 |  1.6145919185935855 |   2407.25 |    0.3163500036792359 |
| 1981-07-30 | 10149.291640417276 |        0.0 |  1.6187864039433986 |  2453.625 |    0.3188320277691965 |
| 1981-07-31 | 10242.510173348142 |        0.0 |  1.6217390101634996 | 2500.3125 |   0.32294609431209864 |
| 1981-08-03 | 10297.727000437446 |        0.0 |   1.623505802671231 | 2546.6875 |   0.32856076915312743 |
| 1981-08-04 | 10421.023325431766 |        0.0 |   1.627450078235627 |    2593.5 |    0.3313308555176924 |
| 1981-08-05 | 10393.572454440577 |  

nil

### Check portfolio record

In [16]:
;; view final portfolio
(view-portfolio)


| :asset | :price |  :aprc | :quantity | :tot-val |
|--------+--------+--------+-----------+----------|
|   cash |    N/A |    N/A |       N/A |   135759 |
|   AAPL | 116.73 | 312.11 |        81 |    25280 |
|      F |  14.87 | 605.11 |         0 |        0 |


nil

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [19]:
(eval-report)

nil

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)